In [ ]:
%env AZURE_OPENAI_API_KEY=
%env AZURE_OPENAI_ENDPOINT=https://haakar-uksouth.openai.azure.com/
%env OPENAI_API_VERSION = 2024-05-01-preview

In [2]:
import os
import json
from openai import AzureOpenAI
client = AzureOpenAI()

In [4]:
file_assistant = client.beta.assistants.create(
    name='File search assistant',
    model='DRIAssistant',
    instructions="You are file search expert who can interpret, summarize the code and explain, share  the code sections",
    tools=[{"type":"file_search"}]    
)

In [21]:
vectorstore = client.beta.vector_stores.create(name="agentscodevectors")
file_paths = ["file_input_company.pdf"]
filestreams = [open(path, "rb") for path in file_paths]


In [22]:
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vectorstore.id,
    files=filestreams
)

In [24]:
file_assistant = client.beta.assistants.update(
    assistant_id=file_assistant.id,
    tool_resources={"file_search":
                    {
                        "vector_store_ids":
                        [
                            vectorstore.id
                        ]
                    }}
)

In [65]:
files = client.files.list()
print(files.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "assistant-sPMPXqiH1VdiuqtnMOWEQT6b",
      "bytes": 2445495,
      "created_at": 1726593280,
      "filename": "file_input_company.pdf",
      "object": "file",
      "purpose": "assistants",
      "status": "processed",
      "status_details": null
    },
    {
      "id": "assistant-aNP0aGqf4R0c5Q8jUGWVHED0",
      "bytes": 2445495,
      "created_at": 1726593234,
      "filename": "file_input_company.pdf",
      "object": "file",
      "purpose": "assistants",
      "status": "processed",
      "status_details": null
    },
    {
      "id": "assistant-yfM01JN5fMNoUz80icd8qZCb",
      "bytes": 2445495,
      "created_at": 1726593122,
      "filename": "file_input_company.pdf",
      "object": "file",
      "purpose": "assistants",
      "status": "processed",
      "status_details": null
    },
    {
      "id": "assistant-SBdhHiJYQHHPZtZmI2fo6Eoc",
      "bytes": 6313,
      "created_at": 1726592904,
      "filename": "forhub.json",
      "object": 

In [25]:
t1 = client.beta.threads.create()

In [68]:
message_file = client.files.create(
  file=open("file_input_company.pdf", "rb"), purpose="assistants"
)

In [75]:
m1 = client.beta.threads.messages.create(
    thread_id=t1.id,
    content="What is the number of zero-day vulnerabilities?",    
    role='user',
    attachments= [
        {
            "file_id": message_file.id,
            "tools":
            [
                {
                    "type": "file_search"
                }
            ]
        }
    ]
)

In [76]:
r1 = client.beta.threads.runs.create(thread_id=t1.id, assistant_id=file_assistant.id)

In [83]:
r1 = client.beta.threads.runs.retrieve(run_id=r1.id, thread_id=t1.id)
r1.status

'completed'

In [84]:
m2 = client.beta.threads.messages.list(thread_id=t1.id)

In [85]:
print(m2.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_U3gU3oy8AccCC8NpptkMdd6T",
      "assistant_id": "asst_tR4R2lWbbtimzGL8ejEosHSi",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The file \"file_input_company.pdf\" contains the Microsoft Digital Defense Report Executive Summary for 2023. It provides an overview of the current cybersecurity landscape, key developments, and strategic responses. Here are some of the key highlights from the document:\n\n1. **Zero-Day Vulnerabilities**: The report discovered 15 new zero-day vulnerabilities in the CODESYS runtime, emphasizing the risks associated with not addressing supply chain vulnerabilities to ensure the security of critical infrastructure and systems【17†source】.\n\n2. **Cybercrime and Security Trends**:\n   - There has been a significant increase in attacks targeting open source software, with an average growth of 742% since 2019.\n   - Human-